# BEN JACKANY CAPSTONE PROJECT
## FLIGHT DELAYS PREDICTIONS

![fig_02](img/fig_02.png)





## Data importing and cleaning

### Libraries

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import missingno as msno
pd.set_option('display.max_columns', None)

#### I downloaded the Reporting Carrier On-Time Performance from the Bureau of transportation statistics website for the years 2019 ( pre CoVid19) and 2020 https://transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr

In [9]:
df = pd.read_csv('../Data/raw_data/2019/1jan2019report.csv')
df.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
0,2019,1,1,16,3,2019-01-16,AA,19805,AA,N150UW,10721,1072102,30721,BOS,"Boston, MA",MA,Massachusetts,13,14100,1410005,34100,PHL,"Philadelphia, PA",PA,Pennsylvania,23,537,528.0,-9.0,0.0,0.0,0001-0559,713,646.0,-27.0,0.0,0.0,0.0,1.0,280.0,NaN
1,2019,1,1,17,4,2019-01-17,AA,19805,AA,N563UW,10721,1072102,30721,BOS,"Boston, MA",MA,Massachusetts,13,14100,1410005,34100,PHL,"Philadelphia, PA",PA,Pennsylvania,23,537,528.0,-9.0,0.0,0.0,0001-0559,713,705.0,-8.0,0.0,0.0,0.0,1.0,280.0,NaN


#### My current data frame consists of flight data for 2019 and 2020. I will use the 2019 data for my machine learning algorithms to predict flight delays and use both the data from 2019 and 2020 to get some useful insights about the impact of the pandemic on domestic flights within the United States. 



### My work will focus on 4 major airlines: American, Delta, South West and United.
### Let's create a dataframe with the above mentioned airlines and export it to a CSV file for further analysis


In [13]:
df_dl = df[df['OP_UNIQUE_CARRIER'] == 'DL']
df_dl.head(5)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
40537,2019,1,1,4,5,2019-01-04,DL,19790,DL,N331DN,12953,1295304,31703,LGA,"New York, NY",NY,New York,22,13487,1348702,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,1055,1052.0,-3.0,0.0,0.0,1000-1059,1327,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0,NaN
40538,2019,1,1,4,5,2019-01-04,DL,19790,DL,N348DN,12953,1295304,31703,LGA,"New York, NY",NY,New York,22,14027,1402702,34027,PBI,"West Palm Beach/Palm Beach, FL",FL,Florida,33,805,806.0,1.0,1.0,0.0,0800-0859,1102,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0,NaN
40539,2019,1,1,4,5,2019-01-04,DL,19790,DL,N970DL,10397,1039707,30397,ATL,"Atlanta, GA",GA,Georgia,34,10693,1069302,30693,BNA,"Nashville, TN",TN,Tennessee,54,826,834.0,8.0,8.0,0.0,0800-0859,834,825.0,-9.0,0.0,0.0,0.0,1.0,214.0,NaN
40540,2019,1,1,4,5,2019-01-04,DL,19790,DL,N970DL,10693,1069302,30693,BNA,"Nashville, TN",TN,Tennessee,54,10397,1039707,30397,ATL,"Atlanta, GA",GA,Georgia,34,918,916.0,-2.0,0.0,0.0,0900-0959,1130,1129.0,-1.0,0.0,0.0,0.0,1.0,214.0,NaN
40541,2019,1,1,4,5,2019-01-04,DL,19790,DL,N538US,13204,1320402,31454,MCO,"Orlando, FL",FL,Florida,33,12478,1247805,31703,JFK,"New York, NY",NY,New York,22,2035,2040.0,5.0,5.0,0.0,2000-2059,2316,2257.0,-19.0,0.0,0.0,0.0,1.0,944.0,NaN


In [8]:
df_aa = df[df['OP_UNIQUE_CARRIER'] == 'AA']
df_aa.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
0,2019,1,1,16,3,2019-01-16,AA,19805,AA,N150UW,...,0001-0559,713,646.0,-27.0,0.0,0.0,0.0,1.0,280.0,NaN
1,2019,1,1,17,4,2019-01-17,AA,19805,AA,N563UW,...,0001-0559,713,705.0,-8.0,0.0,0.0,0.0,1.0,280.0,NaN


In [9]:
df_ua = df[df['OP_UNIQUE_CARRIER'] == 'UA']
df_ua.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
70360,2019,1,1,21,1,2019-01-21,UA,19977,UA,N487UA,...,1800-1859,2058,2104.0,6.0,6.0,0.0,0.0,1.0,1620.0,NaN
70361,2019,1,1,21,1,2019-01-21,UA,19977,UA,N27724,...,0800-0859,1159,1219.0,20.0,20.0,1.0,0.0,1.0,2434.0,NaN


In [10]:
df_wn = df[df['OP_UNIQUE_CARRIER'] == 'WN']
df_wn.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
15112,2019,1,1,1,2,2019-01-01,WN,19393,WN,N229WN,...,2200-2259,2330,2345.0,15.0,15.0,1.0,0.0,1.0,651.0,NaN
15113,2019,1,1,1,2,2019-01-01,WN,19393,WN,N456WN,...,1300-1359,1405,1356.0,-9.0,0.0,0.0,0.0,1.0,651.0,NaN


In [265]:
pd.set_option('display.max_columns', None)

In [11]:
#In [16]: result = pd.concat([df1, df4], ignore_index=True, sort=False)
df_jan = pd.concat([df_dl, df_aa, df_ua, df_wn], ignore_index= True, sort = False)
df_jan.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
0,2019,1,1,4,5,2019-01-04,DL,19790,DL,N331DN,...,1000-1059,1327,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0,NaN
1,2019,1,1,4,5,2019-01-04,DL,19790,DL,N348DN,...,0800-0859,1102,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0,NaN


In [12]:
df_jan.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
0,2019,1,1,4,5,2019-01-04,DL,19790,DL,N331DN,...,1000-1059,1327,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0,NaN
1,2019,1,1,4,5,2019-01-04,DL,19790,DL,N348DN,...,0800-0859,1102,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0,NaN


In [268]:
# #Relevant columns to keep for our analysis: 

# # 'YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
#        
#        'OP_UNIQUE_CARRIER'
#        'ORIGIN',
#        'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
#        
#         'DEST', 'DEST_STATE_ABR',
#         'DEST_STATE_NM', 'CRS_DEP_TIME',
#        'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
#        'DEP_DELAY_GROUP', 'ARR_TIME', 'ARR_DELAY',
#        'ARR_DELAY_NEW', 'ARR_DEL15', 'CANCELLED', 'FLIGHTS', 'DISTANCE',
#   


In [30]:
df_jan['CANCELLED'].sum()

6485.0

In [14]:
df_jan.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE,Unnamed: 40
0,2019,1,1,4,5,2019-01-04,DL,19790,DL,N331DN,...,1000-1059,1327,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0,NaN
1,2019,1,1,4,5,2019-01-04,DL,19790,DL,N348DN,...,0800-0859,1102,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0,NaN


In [15]:
df_jan_keep = df_jan[['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE','OP_UNIQUE_CARRIER',
        'ORIGIN','ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
        'DEST', 'DEST_STATE_ABR','DEST_STATE_NM', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
        'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'ARR_DEL15', 'CANCELLED', 'FLIGHTS','DISTANCE']]

In [16]:
df_jan_keep.head(2)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,...,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE
0,2019,1,1,4,5,2019-01-04,DL,LGA,NY,New York,...,-3.0,0.0,0.0,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0
1,2019,1,1,4,5,2019-01-04,DL,LGA,NY,New York,...,1.0,1.0,0.0,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0


In [24]:
df_jan_keep.dropna(inplace= True)

<ipython-input-24-b0d9234ee54f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_keep.dropna(inplace= True)


In [25]:
df_jan_keep.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,...,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,FLIGHTS,DISTANCE
0,2019,1,1,4,5,2019-01-04,DL,LGA,NY,New York,...,-3.0,0.0,0.0,1237.0,-50.0,0.0,0.0,0.0,1.0,1020.0
1,2019,1,1,4,5,2019-01-04,DL,LGA,NY,New York,...,1.0,1.0,0.0,1058.0,-4.0,0.0,0.0,0.0,1.0,1035.0
2,2019,1,1,4,5,2019-01-04,DL,ATL,GA,Georgia,...,8.0,8.0,0.0,825.0,-9.0,0.0,0.0,0.0,1.0,214.0
3,2019,1,1,4,5,2019-01-04,DL,BNA,TN,Tennessee,...,-2.0,0.0,0.0,1129.0,-1.0,0.0,0.0,0.0,1.0,214.0
4,2019,1,1,4,5,2019-01-04,DL,MCO,FL,Florida,...,5.0,5.0,0.0,2257.0,-19.0,0.0,0.0,0.0,1.0,944.0


In [26]:
df_jan2019 = df_jan_keep

In [27]:
df_jan2019.shape

(302118, 25)

In [28]:
#df_jan2019.to_csv('df_jan2019.csv', index = False)

In [29]:
df_jan2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302118 entries, 0 to 309079
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   YEAR               302118 non-null  int64  
 1   QUARTER            302118 non-null  int64  
 2   MONTH              302118 non-null  int64  
 3   DAY_OF_MONTH       302118 non-null  int64  
 4   DAY_OF_WEEK        302118 non-null  int64  
 5   FL_DATE            302118 non-null  object 
 6   OP_UNIQUE_CARRIER  302118 non-null  object 
 7   ORIGIN             302118 non-null  object 
 8   ORIGIN_STATE_ABR   302118 non-null  object 
 9   ORIGIN_STATE_NM    302118 non-null  object 
 10  DEST               302118 non-null  object 
 11  DEST_STATE_ABR     302118 non-null  object 
 12  DEST_STATE_NM      302118 non-null  object 
 13  CRS_DEP_TIME       302118 non-null  int64  
 14  DEP_TIME           302118 non-null  float64
 15  DEP_DELAY          302118 non-null  float64
 16  DE

### I saved the 2019 and 2020 datasets as well as January and february of 2021( the latest available data as of April 2021 was the data of february of 2021)